# Use SPSS to predict customer churn with `ibm-watson-machine-learning`

This notebook contains steps to deploy sample SPSS stream, and start scoring new data. 

Some familiarity with bash is helpful. This notebook uses Python 3.7.

You will use a data set, **Telco Customer Churn**, which details anonymous customer data from a telecommunication company. Use the details of this data set to predict customer churn which is very critical to business as it's easier to retain existing customers rather than acquiring new ones.

## Learning goals

The learning goals of this notebook are:

-  Working with the Watson Machine Learning instance
-  Online deployment of SPSS model
-  Scoring data using deployed model


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Model upload](#upload) 
3.	[Web service creation](#deploy)
4.	[Scoring](#score)
5.  [Clean up](#cleanup)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [2]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First, create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Upload model

In this section you will learn how to upload the model to the Cloud.


**Action**: Download sample SPSS model from git project using wget.

In [6]:
import os
from wget import download

sample_dir = 'spss_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename=os.path.join(sample_dir, 'customer-satisfaction-prediction.str')
if not os.path.isfile(filename):
    filename = download('https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/models/spss/customer_satisfaction/model/customer-satisfaction-prediction.str',\
                             out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


Store SPSS sample model in your Watson Machine Learning instance.

In [ ]:
client.software_specifications.list()

In [7]:
sw_spec_uid = client.software_specifications.get_uid_by_name("spss-modeler_18.2")

model_meta_props = {
    client.repository.ModelMetaNames.NAME: "SPSS customer satisfaction model",
    client.repository.ModelMetaNames.TYPE: "spss-modeler_18.2",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
}

model_details = client.repository.store_model(filename, model_meta_props)

**Note:** You can see that model is successfully stored in Watson Machine Learning Service.

In [ ]:
client.repository.list_models()

<a id="deployment"></a>
## 3. Create online deployment
You can use commands bellow to create online deployment for stored model (web service).

In [8]:
model_uid = client.repository.get_model_uid(model_details)

deployment = client.deployments.create(
    artifact_uid=model_uid,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "SPSS deployment",
        client.deployments.ConfigurationMetaNames.ONLINE:{}}
)



#######################################################################################

Synchronous deployment creation for uid: 'c2219084-0a3a-48fc-9afb-663b566fcb58' started

#######################################################################################


initializing.....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='c600e46d-eb55-4b2d-be81-09a79aa428a3'
------------------------------------------------------------------------------------------------




<a id="scoring"></a>
## 4. Scoring

You can send new scoring records to web-service deployment using `score` method.

In [9]:
deployment_id = client.deployments.get_id(deployment)

scoring_data = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["customerID","gender","SeniorCitizen","Partner","Dependents","tenure","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod","MonthlyCharges","TotalCharges","Churn","SampleWeight"],
        "values":[["3638-WEABW","Female",0,"Yes","No",58,"Yes","Yes","DSL","No","Yes","No","Yes","No","No","Two year","Yes","Credit card (automatic)",59.9,3505.1,"No",2.768]]
    }]
}

predictions = client.deployments.score(deployment_id, scoring_data)
print(predictions)

{'predictions': [{'fields': ['customerID', 'Churn', 'Predicted Churn', 'Probability of Churn'], 'values': [['3638-WEABW', 'No', 'No', 0.0526309571556145]]}]}


As we can see this sample telco customer is satisfied ("Predicted Churn", "No").

<a id="cleanup"></a>
## 5. Clean up   

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

see the steps in this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for SPSS model deployment and scoring. Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Lukasz Cmielowski, PhD,** is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.